In [1]:
import os
import zipfile
import json
import random
from tqdm import tqdm
import plotly
import plotly.express as px
import plotly.graph_objects as go

import numpy as np
import pandas as pd

from helpers import tokenize_and_format, flat_accuracy

import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup

In [2]:
random.seed(0)
np.random.seed(0)

torch.manual_seed(0)
torch.use_deterministic_algorithms(False)
# Confirm that the GPU is detected

assert torch.cuda.is_available()

# Get the GPU device name.
device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
device = torch.device("cuda")

Found device: NVIDIA GeForce RTX 2060 with Max-Q Design, n_gpu: 1


### Load data

In [3]:
authors_train = torch.load("Data/BERT output/authors_train.pt")
authors_val = torch.load("Data/BERT output/authors_val.pt")
authors_test = torch.load("Data/BERT output/authors_test.pt")

abstract_train = torch.load("Data/BERT output/abstract_train.pt")
abstract_val = torch.load("Data/BERT output/abstract_val.pt")
abstract_test = torch.load("Data/BERT output/abstract_test.pt")

In [4]:
training_inputs = torch.cat([authors_train, abstract_train], dim = 1)
validation_inputs = torch.cat([authors_val, abstract_val], dim = 1)
test_inputs = torch.cat([authors_test, abstract_test], dim = 1)

In [5]:
with open("Data/Raw data/training_data.jsonl", "r") as f:
    training_data = json.load(f)
    
with open("Data/Raw data/validation_data.jsonl", "r") as f:
    validation_data = json.load(f)
    
with open("Data/Raw data/test_data.jsonl", "r") as f:
    test_data = json.load(f)

with open("Data/Metadata/label_string_to_ID.jsonl", "r") as f:
    label_string_to_ID = json.load(f)
    
with open("Data/Metadata/label_ID_to_string.jsonl", "r") as f:
    label_ID_to_string = json.load(f)

training_label_strings = []
validation_label_strings = []
test_label_strings = []

for training_example in training_data:
    training_label_strings.append(training_example[1])
    
for validation_example in validation_data:
    validation_label_strings.append(validation_example[1])
    
for test_example in test_data:
    test_label_strings.append(test_example[1])
    
training_label_IDs = []
validation_label_IDs = []
test_label_IDs = []

for training_label_string in training_label_strings:
    training_label_IDs.append(label_string_to_ID[training_label_string])
    
for validation_label_string in validation_label_strings:
    validation_label_IDs.append(label_string_to_ID[validation_label_string])
    
for test_label_string in test_label_strings:
    test_label_IDs.append(label_string_to_ID[test_label_string])
    
training_label_IDs = torch.tensor(training_label_IDs)
validation_label_IDs = torch.tensor(validation_label_IDs)
test_label_IDs = torch.tensor(test_label_IDs)

### Train the deep classifier

In [7]:
class DeepClassifier(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(DeepClassifier, self).__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, output_dim)
        
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()
        
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        hidden = self.linear1(x)
        hidden = self.relu(hidden)
        
        hidden = self.dropout(hidden)
        
        hidden = self.linear2(hidden)
        out = self.softmax(hidden)
        
        return out
    

input_dim = training_inputs.shape[1]
hidden_dim = 128
output_dim = len(label_string_to_ID)

model = DeepClassifier(input_dim, hidden_dim, output_dim)




# Tell pytorch to run this model on the GPU.
model.cuda()

criterion = nn.CrossEntropyLoss()

hyperparameter_config_iter = 1

save_path = "Saved models/Authors and abstract/Hyperparameter configuration " + str(hyperparameter_config_iter)

if(os.path.exists(save_path)):
    raise Exception("ERROR! Hyperparameter config " + str(hyperparameter_config_iter))

else:
    os.makedirs(save_path)
    os.makedirs(save_path + "/Plots")


# Fine-tuning hyperparameters

batch_size = 16

optimizer = torch.optim.Adam(model.parameters())
epochs = 200

hyperparameter_dict = dict()
hyperparameter_dict['batch_size'] = batch_size
hyperparameter_dict['epochs'] = epochs


with open(save_path + "/Hyperparameters.json", 'w') as f:
    json.dump(hyperparameter_dict, f)

def save(model, optimizer, output_path):
    torch.save({'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict()}, output_path)

# function to get validation accuracy
def get_performance(data_set):
    # Put the model in evaluation mode
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0

    num_batches = int(len(data_set)/batch_size) + 1

    total_correct = 0

    for i in range(num_batches):

        end_index = min(batch_size * (i+1), len(data_set))
        
        if len(data_set[0]) == 0: continue

        input_id_tensors = data_set[0][i*batch_size:end_index]
        label_tensors = data_set[1][i*batch_size:end_index]

        # Move tensors to the GPU
        b_input_ids = input_id_tensors.to(device)
        b_labels = label_tensors.to(device)

        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            b_outputs = model(b_input_ids)
            
            loss = criterion(b_outputs, b_labels)

            logits = b_outputs

            # Accumulate the validation loss.
            total_eval_loss += loss.item()

            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            # Calculate the number of correctly labeled examples in batch
            pred_flat = np.argmax(logits, axis=1).flatten()
            labels_flat = label_ids.flatten()
            num_correct = np.sum(pred_flat == labels_flat)
            total_correct += num_correct
        
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_correct / len(data_set)
    return avg_val_accuracy

# training loop

max_val_acc = -1

metric_vs_epoch = dict()

epoch_list = []
training_loss_list = []
training_acc_list = []

val_acc_list = []

# For each epoch...
for epoch_i in range(0, epochs):
    
    epoch_list.append(epoch_i + 1)
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode.
    model.train()

    # For each batch of training data...
    num_batches = int(training_inputs.shape[0]/batch_size) + 1

    for i in tqdm(range(num_batches)):
        
        end_index = min(batch_size * (i+1), training_inputs.shape[0])

        b_input_ids = training_inputs[i*batch_size:end_index]
        b_labels = training_label_IDs[i*batch_size:end_index]

        if len(b_input_ids) == 0: continue

        # Move tensors to the GPU
        b_input_ids = b_input_ids.to(device)
        b_labels = b_labels.to(device)

        # Clear the previously calculated gradient
        model.zero_grad()

        # Perform a forward pass (evaluate the model on this training batch).
        b_outputs = model(b_input_ids)
        
        loss = criterion(b_outputs, b_labels)
        
        #logits = outputs.logits

        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Update parameters and take a step using the computed gradient.
        optimizer.step()
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set. Implement this function in the cell above.
    
    training_acc = get_performance([training_inputs, training_label_IDs])
    val_acc = get_performance([validation_inputs, validation_label_IDs])
    
    print(f"Total loss: {total_train_loss}")
    print(f"Validation accuracy: {val_acc}")
    
    val_acc_list.append(val_acc)
    training_acc_list.append(training_acc)
    
    training_loss_list.append(total_train_loss)
    
    if(val_acc > max_val_acc):
        
        max_val_acc = val_acc
        
        torch.save(model.state_dict(), save_path + "/best validation accuracy model.model")
        save(model, optimizer, save_path + "/best validation accuracy.modelState")
    
    
print("")
print("Training complete!")            

metric_vs_epoch["Epochs"] = epoch_list
metric_vs_epoch["Training loss"] = training_loss_list
metric_vs_epoch["Training accuracy"] = training_acc_list
metric_vs_epoch["Validation accuracy"] = val_acc_list

with open(save_path + "/Plots/Plot data.json", 'w') as f:
    json.dump(metric_vs_epoch, f)

metric_vs_epoch_df = pd.DataFrame(metric_vs_epoch, columns = ["Epochs", "Training loss", "Training accuracy", "Validation accuracy"])

fig = px.line(metric_vs_epoch_df, x='Epochs', y="Training loss", title="Training loss vs epochs")
plotly.offline.plot(fig, filename = save_path + "/Plots/Training loss.html")

accuracy_vs_epoch = dict()
accuracy_vs_epoch["Epochs"] = epoch_list + epoch_list
accuracy_vs_epoch["Accuracy"] = training_acc_list + val_acc_list
accuracy_vs_epoch["Dataset"] = ["Training"]*len(training_acc_list) + ["Validation"]*len(val_acc_list)

accuracy_vs_epoch_df = pd.DataFrame(accuracy_vs_epoch, columns = ["Epochs", "Accuracy", "Dataset"])

fig = px.line(accuracy_vs_epoch_df, x='Epochs', y='Accuracy', color='Dataset', markers=True, title="Training/Validation accuracy vs epochs")
plotly.offline.plot(fig, filename = save_path + "/Plots/Accuracy.html")

with open(save_path + "/Best validation accuracy.txt", 'w') as f:
    f.write("Best validation accuracy: " + str(max_val_acc))


======== Epoch 1 / 200 ========
Training...


  0%|                                                                                           | 0/87 [00:00<?, ?it/s]C:\Users\arbaa\AppData\Local\Temp/ipykernel_1260/2077003762.py:20: UserWarning:

Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.

100%|█████████████████████████████████████████████████████████████████████████████████| 87/87 [00:00<00:00, 176.83it/s]


Total loss: 192.68827724456787
Validation accuracy: 0.5

======== Epoch 2 / 200 ========
Training...


100%|█████████████████████████████████████████████████████████████████████████████████| 87/87 [00:00<00:00, 480.67it/s]


Total loss: 181.2205319404602
Validation accuracy: 0.5

======== Epoch 3 / 200 ========
Training...


...

Total loss: 180.8620011806488
Validation accuracy: 0.5

======== Epoch 199 / 200 ========
Training...


100%|█████████████████████████████████████████████████████████████████████████████████| 87/87 [00:00<00:00, 591.85it/s]


Total loss: 180.86199069023132
Validation accuracy: 0.5

======== Epoch 200 / 200 ========
Training...


100%|█████████████████████████████████████████████████████████████████████████████████| 87/87 [00:00<00:00, 568.64it/s]


Total loss: 180.87264752388
Validation accuracy: 0.5

Training complete!
